# **Databases Viewer**

<span style="font-family: times, Optima; font-size:11pt; color:black;">
    
This notebook serves as a viewer of the wind, wave and tidal databases available in Upolu and Tongatapu Islands:<br>

**Sea level (m)** : The near-real-time sea level is downloaded in a bunch of tidal gauges all along the world, using the data provided by the UNESCO Intergovernmental Oceanographic Commission (IOC) at http://www.ioc-sealevelmonitoring.org/list.php?showall=a&output=general&order=country&dir=asc. The historical sea level (1992, 2021) is also downloaded from BOM (Bureau of Meteorology), available at a tide gauge facility (http://www.bom.gov.au/pacific/projects/pslm/index.shtml)<br>

**Waves** : The significant wave heigh (Hs), peak period (Tp) and mean direction (Dir) are obtained from the CSIRO global wave hindcast CAWR (1979-2020) in a grid resolution of 0.4 x 0.4°.<br>

**Wind** : Hourly Time-Series of wind from the NCEP Climate Forecast System Reanalysis (CFSR) with data information from January 1979 to December 2010.<br>

**Precipitation** : Historical series of precipitation (mm/day) (1982-2019) from the CFS (Climate Forecast System)

**Storm Surge** : A high resolution Storm Surge (m) dataset from the Coastal Dataset for the Evaluation of Climate Impact (CoDEC)

<span>

Table of contents:
* [1. Load databases](#1)
* [2. Location map](#2)
* [3. Data visualization](#3)
    * [3.1 Sea level](#31)
        * [3.1.1 Near-real-time IOC](#311)
        * [3.1.2 Historical BOM](#312)
        * [3.1.3 Storage output data](#313)
    * [3.2 Waves](#32)
        * [3.2.1 Time series visualization of Station ID](#321)
        * [3.2.2 Time frame animation](#322)
        * [3.2.3 Storage output data](#323)
    * [3.3 Wind](#32)
        * [3.3.1 Time series visualization of Station ID](#331)
        * [3.3.2 Time frame animation](#332)
        * [3.3.3 Storage output data](#333)    
    * [3.4 Precipitation](#332)
        * [3.4.1 Time series visualization of Station ID](#341)
        * [3.4.2 Time frame animation](#342)
        * [3.4.3 Storage output data](#343)  
    * [3.5 Storm Surge](#332)
        * [3.5.1 Time series visualization of Station ID](#341)
        * [3.5.2 Time frame animation](#342)
        * [3.5.3 Storage output data](#343) 
<hr size="5"/>

In [ ]:
import os
import os.path as op
import sys

# basic import
import pandas as pd
import xarray as xr
import numpy as np
import datetime

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

# append ddviewer to path
sys.path.insert(0, op.join(os.getcwd(), '..', '..'))

# dependencies
from lib.wget import download_ioc_stations, download_station_by_code
from lib.plot import *
from lib.config import *

## 1. Load databases <a class="anchor" id="1"></a>

`p_data` is the folder path where the individual databases are located and in `p_storage` the output files of this notebook will be storage

In [ ]:
p_data = op.join(os.getcwd(), '..', 'data', 'databases')
p_storage = op.join(p_data, '..', '..', 'storage')

In [ ]:
# Download sea-level stations inside selected region
stations_ioc = download_ioc_stations(region)
stations_bom = xr.open_dataset(op.join(p_data, 'historical_BOM_data', 'historical_BOM_stations.nc'))

In [ ]:
# Load databases (waves, wind, prec, ss)
p_waves = op.join(p_data, 'Waves_CSIRO.nc')
p_wind = op.join(p_data, 'Wind_CFSR.nc')
p_prec = op.join(p_data, 'P_CFS.nc')
p_codec = op.join(p_data, 'SS_CODEC.nc')
p_pressure = op.join(p_data, 'SLPS_1979_2019.nc')

waves = xr.open_dataset(p_waves)
wind = xr.open_dataset(p_wind)
prec = xr.open_dataset(p_prec)
codec = xr.open_dataset(p_codec)
pressure = xr.open_dataset(p_pressure)

In [ ]:
# Extract lat-lon coordinates-mesh
latt_wvs, lonn_wvs = np.meshgrid(waves.latitude, waves.longitude)
latt_wnd, lonn_wnd = np.meshgrid(wind.lat, wind.lon)
latt_prc, lonn_prec = np.meshgrid(prec.lat, prec.lon)
latt_pres, lon_pres = np.meshgrid(pressure.latitude, pressure.longitude)

df_wvs = pd.DataFrame({'lat':np.concatenate(latt_wvs), 'lon':np.concatenate(lonn_wvs),})
df_wnd = pd.DataFrame({'lat':np.concatenate(latt_wnd), 'lon':np.concatenate(lonn_wnd),})
df_prec = pd.DataFrame({'lat':np.concatenate(latt_prc),'lon':np.concatenate(lonn_prec),})
df_pres = pd.DataFrame({'lat':np.concatenate(latt_pres),'lon':np.concatenate(lon_pres),})

## Location map <a class="anchor" id="2"></a>

An interactive map is displayed for the user to identify the stations in the database. The top panel allows items to be deselected for better visualisation.

In [ ]:
px_databases(df_wnd, df_wvs, df_prec, stations_ioc, stations_bom, codec, df_pres)

## 2. Data visualization <a class="anchor" id="3"></a>

### Sea level <a class="anchor" id="31"></a>

#### Near-real-time IOC<a class="anchor" id="311"></a>

In [ ]:
# download selected station
code = 'nkfa' 
station_data = download_station_by_code(code, '1 week')

In [ ]:
fig = px.line(station_data.rename(columns={'aqu(m)':'Level(m)'}))
fig.update_layout(width=1000,height=300,)
fig.show()

#### Historical BOM<a class="anchor" id="312"></a>

In [ ]:
# Download BOM station data
station_id = 'IDO70012'            
ds_i = stations_bom.sel(ID_CODE=station_id)

**- Run historical period (1992-2021)**

In [ ]:
df_subset_sl = ds_i

**- Select a period of data (a month, a year)**

In [ ]:
# initial date (year, month, day)
d1, m1, y1 = 1, 4, 2000

# final date 
d2, m2, y2,  = 1, 5, 2000

df_subset_sl = ds_i.sel(Date=slice(datetime.datetime(y1,m1,d1),  datetime.datetime(y2,m2,d2)))

In [ ]:
#%matplotlib
df_subset_sl['Sea level'].plot(
    color=colors[0],
    figsize=(10,3)
)
plt.show()

#### Storage output data<a class="anchor" id="313"></a>

In [ ]:
# storage .csv
df_subset_sl.to_dataframe()[['Sea level']].to_csv(op.join(p_storage, 'Sea_level_{0}.csv'.format(ds_i.Country.values)))

---
<br>

### Waves <a class="anchor" id="32"></a>

In [ ]:
ID_wave = 1139

#### Time series visualization of Station ID<a class="anchor" id="321"></a>

In [ ]:
dfi = df_wvs.loc[[ID_wave]]

wvs_i = waves.sel(latitude=dfi.lat.values, longitude=dfi.lon.values).squeeze()
df_wvs_i = wvs_i.to_dataframe()[['hs', 'fp', 'dir']]
df_wvs_i['tp'] = 1/df_wvs_i['fp']

**- Run historical period (1979-2020)**

In [ ]:
df_subset_waves = df_wvs_i

**- Select a period of data (a month, a year)**

In [ ]:
# initial date (year, month, day)
d1, m1, y1 = 2, 4, 2020

# final date 
d2, m2, y2,  = 10, 4, 2020

df_subset_waves = df_wvs_i[datetime.datetime(y1,m1,d1) : datetime.datetime(y2,m2,d2)]

**Plot time series**

In [ ]:
#%matplotlib
df_subset_waves[['hs', 'tp', 'dir']].plot(
    figsize=(10, 3),
    style=['-','-', '.'],
    ms=2,
    color=[colors[0],colors[1],colors[2]],
    subplots=True, 
    layout=(3,1),
)
plt.show()

In [ ]:
df_subset_waves = df_subset_waves.replace([np.inf, -np.inf], np.nan)
fig, axs = plt.subplots(1, 3, figsize=(17,5))
axs[0].hist(df_subset_waves['hs'], alpha=0.7, density=1, rwidth=0.95, color=colors[0])
axs[1].hist(df_subset_waves['tp'], alpha=0.7, density=1, rwidth=0.95, color=colors[1])
axs[2].hist(df_subset_waves['dir'], alpha=0.7, density=1,  rwidth=0.95, color=colors[2])
axs[0].set_xlabel('Hs (m)')
axs[1].set_xlabel('Tp (s)')
axs[2].set_xlabel('Dir (º)')
plt.show()

#### Time frame animation<a class="anchor" id="322"></a>

In [ ]:
# Choose between dataset variables ('hs', 'tp', 'dir')
var = ['hs']

# custom colormap
if var == ['dir']: colormap = px.colors.cyclical.HSV
else: colormap = px.colors.sequential.Hot_r

waves_subset = waves.sel(time=slice(datetime.datetime(y1,m1,d1), datetime.datetime(y2,m2,d2)))
waves_subset = waves_subset[var].to_dataframe().reset_index()
waves_subset['time'] = waves_subset['time'].astype(str).tolist()

In [ ]:
fig = px.scatter_mapbox(waves_subset, lat="latitude", lon="longitude", 
                        color_continuous_scale=colormap,
                        color=var[0], 
                        zoom=3.6, height=550, width=500, 
                        animation_frame="time",
                        size_max=120,
                        range_color=(np.nanmin(waves_subset[var].values), np.nanmax(waves_subset[var].values))
)
fig.update_traces(marker=dict(size=12))
fig.update_layout(
    margin={"r":2,"t":0,"l":0,"b":0}, 
    width=900, height=500,
    mapbox_style="open-street-map"

)
fig.show()

#### Storage output data <a class="anchor" id="323"></a>

In [ ]:
# storage .csv
df_subset_waves.to_csv(op.join(p_storage, 'Waves_{0}.csv'.format(ID_wave)))

---
<br>

### Wind <a class="anchor" id="33"></a>

In [ ]:
ID_wind = 616

#### Time series visualization of Station ID <a class="anchor" id="331"></a>

In [ ]:
dfii = df_wnd.loc[[ID_wind]]

wnd_i = wind.sel(lat=dfii.lat.values, lon=dfii.lon.values).squeeze()
df_wnd_i = wnd_i.to_dataframe()[['w', 'wdir']]

**- Run historical period (1979-2021)**

In [ ]:
df_subset_wind = df_wnd_i

**- Select a period of data (a month, a year)**

In [ ]:
# initial date (year, month, day)
d1, m1, y1 = 2, 4, 2020

# final date 
d2, m2, y2,  = 10, 4, 2020

df_subset_wind = df_wnd_i[datetime.datetime(y1,m1,d1) : datetime.datetime(y2,m2,d2)]

In [ ]:
#%matplotlib
df_subset_wind.plot(
    figsize=(10,3),
    style=['-', '.'],
    ms=7,
    color=[colors[0],colors[1]],
    subplots=True, 
)
plt.show()

In [ ]:
#%matplotlib
df_subset_wind = df_subset_wind.replace([np.inf, -np.inf], np.nan)
fig, axs = plt.subplots(1, 2, figsize=(12,5))
axs[0].hist(df_subset_wind['w'], alpha=0.7, density=1, rwidth=0.95, color=colors[0])
axs[1].hist(df_subset_wind['wdir'], alpha=0.7, density=1, rwidth=0.95, color=colors[2])
axs[0].set_xlabel('W (m/s)')
axs[1].set_xlabel('Wdir(º)')
plt.show()

#### Time frame animation<a class="anchor" id="332"></a>

In [ ]:
# Choose between dataset variables ('w', 'wdir')
var = ['wdir']

# custom colormap
if var == ['wdir']: colormap = px.colors.cyclical.HSV
else: colormap = px.colors.sequential.dense

wind_subset = wind.sel(time=slice(datetime.datetime(y1,m1,d1), datetime.datetime(y2,m2,d2)))
wind_subset = wind_subset[var].to_dataframe().reset_index()
wind_subset['time'] = wind_subset['time'].astype(str).tolist()

In [ ]:
fig = px.scatter_mapbox(wind_subset, lat="lat", lon="lon", 
                        color_continuous_scale=colormap,
                        color=var[0], 
                        zoom=3.6, height=550, width=500, 
                        animation_frame="time",
                        size_max=120,
                        range_color=(np.nanmin(wind_subset[var].values), np.nanmax(wind_subset[var].values))
)
fig.update_traces(marker=dict(size=12))
fig.update_layout(
    margin={"r":2,"t":0,"l":0,"b":0}, 
    width=900, height=500,
    mapbox_style="open-street-map"

)
fig.show()

#### Storage output data <a class="anchor" id="333"></a>

In [ ]:
# storage .csv
df_subset_wind.to_csv(op.join(p_storage, 'Wind_{0}.csv'.format(ID_wind)))

---
<br>

### Precipitation <a class="anchor" id="34"></a>

In [ ]:
ID_prec = 36

#### Time series visualization of Station ID <a class="anchor" id="341"></a>

In [ ]:
dfii_1 = df_prec.loc[[ID_prec]]

prec_i = prec.sel(lat=dfii_1.lat, lon=dfii_1.lon).squeeze()
df_prec_i = prec_i.to_dataframe()[['pratel']].rename(columns={'pratel':'Precipitation'})

**- Run historical period (1982-2019)**

In [ ]:
df_subset_prec = df_prec_i

**- Select a period of data (a month, a year)**

In [ ]:
# initial date (year, month, day)
d1, m1, y1 = 4, 4, 2019

# final date 
d2, m2, y2,  = 10, 4, 2020

df_subset_prec = df_prec_i[datetime.datetime(y1,m1,d1) : datetime.datetime(y2,m2,d2)]

**Plot**

In [ ]:
fig = px.line(df_subset_prec)
fig.update_layout(width=1000,height=300,)
fig.show()

#### Time frame animation<a class="anchor" id="342"></a>

In [ ]:
var = ['pratel']

prec_subset = prec.sel(time=slice(datetime.datetime(y1,m1,d1), datetime.datetime(y2,m2,d2)))
prec_subset = prec_subset[var].to_dataframe().reset_index()
prec_subset['time'] = prec_subset['time'].astype(str).tolist()

In [ ]:
fig = px.scatter_mapbox(prec_subset, lat="lat", lon="lon", 
                        color_continuous_scale=px.colors.sequential.Magma_r,
                        color=var[0], 
                        zoom=3.6, height=550, width=500, 
                        animation_frame="time",
                        size_max=300,
                        range_color=(np.nanmin(prec_subset[var].values), np.nanmax(prec_subset[var].values))
)
fig.update_traces(marker=dict(size=12))
fig.update_layout(
    margin={"r":2,"t":0,"l":0,"b":0}, 
    width=900, height=500,
    mapbox_style="open-street-map"

)
fig.show()

#### Storage output data <a class="anchor" id="342"></a>

In [ ]:
# storage .csv
df_subset_prec.to_csv(op.join(p_storage, 'Precipitation_{0}.csv'.format(ID_prec)))

---
<br>

### Storm Surge <a class="anchor" id="35"></a>

In [ ]:
ID_ss = 9

#### Time series visualization of Station ID <a class="anchor" id="341"></a>

In [ ]:
df_cod = codec.sel(Station=ID_ss).squeeze()
df_ss = df_cod.to_dataframe()[['SS']]

**- Run historical period (1979-2018)**

In [ ]:
df_subset_ss = df_ss

**- Select a period of data (a month, a year)**

In [ ]:
# initial date (year, month, day)
d1, m1, y1 = 4, 4, 2015

# final date 
d2, m2, y2,  = 10, 4, 2015

df_subset_ss = df_ss[datetime.datetime(y1,m1,d1) : datetime.datetime(y2,m2,d2)]

**Plot**

In [ ]:
fig = px.line(df_subset_ss)
fig.update_layout(width=1000,height=300,)
fig.show()

#### Time frame animation<a class="anchor" id="342"></a>

In [ ]:
var = ['SS']

ss_subset = codec.sel(Date=slice(datetime.datetime(y1,m1,d1), datetime.datetime(y2,m2,d2)))
ss_subset = ss_subset[var].to_dataframe().reset_index()
ss_subset['time'] = ss_subset['Date'].astype(str).tolist()

In [ ]:
fig = px.scatter_mapbox(ss_subset, lat="Lat", lon="Lon", 
                        color_continuous_scale=px.colors.sequential.RdBu_r,
                        color=var[0], 
                        zoom=3.6, height=550, width=500, 
                        animation_frame="time",
                        size_max=300,
                        range_color=(np.nanmin(ss_subset[var].values), np.nanmax(ss_subset[var].values))
)
fig.update_traces(marker=dict(size=12))
fig.update_layout(
    margin={"r":2,"t":0,"l":0,"b":0}, 
    width=900, height=500,
    mapbox_style="open-street-map"

)
fig.show()

#### Storage output data <a class="anchor" id="342"></a>

In [ ]:
# storage .csv
df_subset_ss.to_csv(op.join(p_storage, 'StormSurge_{0}.csv'.format(ID_ss)))

### Pressure <a class="anchor" id="36"></a>

In [ ]:
ID_pres = 342

#### Time series visualization of Station ID <a class="anchor" id="341"></a>

In [ ]:
df_p = df_pres.loc[ID_pres].squeeze()

pres_i = pressure.sel(latitude=df_p.lat, longitude=df_p.lon).squeeze()
df_pres_i = pres_i.to_dataframe()[['SLP']]

**- Run historical period (1979-2018)**

In [ ]:
df_subset_pres = df_pres_i

**- Select a period of data (a month, a year)**

In [ ]:
# initial date (year, month, day)
d1, m1, y1 = 4, 4, 2015

# final date 
d2, m2, y2,  = 10, 4, 2015

df_subset_pres = df_pres_i[datetime.datetime(y1,m1,d1) : datetime.datetime(y2,m2,d2)]

**Plot**

In [ ]:
fig = px.line(df_subset_pres)
fig.update_layout(width=1000,height=300,)
fig.show()

#### Time frame animation<a class="anchor" id="342"></a>

In [ ]:
var = ['SLP']

pr_subset = pressure.sel(time=slice(datetime.datetime(y1,m1,d1), datetime.datetime(y2,m2,d2)))
pr_subset = pr_subset[var].to_dataframe().reset_index()
pr_subset['time'] = pr_subset['time'].astype(str).tolist()

fig = px.scatter_mapbox(pr_subset, lat="latitude", lon="longitude", 
                        color_continuous_scale=px.colors.sequential.RdBu_r,
                        color=var[0], 
                        zoom=3.6, height=550, width=500, 
                        animation_frame="time",
                        size_max=300,
                        range_color=(np.nanmin(pr_subset[var].values), np.nanmax(pr_subset[var].values))
)
fig.update_traces(marker=dict(size=12))
fig.update_layout(
    margin={"r":2,"t":0,"l":0,"b":0}, 
    width=900, height=500,
    mapbox_style="open-street-map"

)
fig.show()

#### Storage output data <a class="anchor" id="342"></a>

In [ ]:
# storage .csv
df_subset_pres.to_csv(op.join(p_storage, 'Pressure_{0}.csv'.format(ID_pres)))